In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, Lasso, Ridge, BayesianRidge,ElasticNet
import patsy as ps
from sklearn.model_selection import KFold 

In [ ]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')
df_train

In [ ]:
df_variable = pd.read_csv('VariablesUsed.csv')
df_variable.groupby(['used'])['used'].count()

In [ ]:
df_train = df_train.drop(['CLTV','MIP','MIT'],axis = 1)
df_test = df_test.drop(['CLTV','MIP','MIT'],axis = 1)
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
X_train = df_train.iloc[:,1:-2]
X_test = df_test.iloc[:,1:]
Y_train = df_train['NMONTHS']
Y_train = Y_train.to_numpy()
names_train = X_train.columns.values
names_test = X_test.columns.values

In [ ]:
categorical_cols = ['CHANNEL','SELLER','FTHBI','LP','PTYPE','OCCSTAT','STATE','RMI']

In [ ]:
for col in X_train.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(X_train[col].unique())):
            a[X_train[col].unique()[i]]=i
        work=X_train[col].map(a)
        X_train[col]=work
#encoded = list(X_train.columns)

for col in X_test.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(X_test[col].unique())):
            a[X_test[col].unique()[i]]=i
        work=X_test[col].map(a)
        X_test[col]=work
#encoded = list(X_test.columns)

In [ ]:
Date = ['MRP','ODATE','FPD']
for i in Date:
    X_train[i]= pd.to_datetime(X_train[i])
    X_test[i]= pd.to_datetime(X_test[i])
    X_train[i] =(X_train[i]-min(X_train[i])).dt.days
    X_test[i] =(X_test[i]-min(X_test[i])).dt.days

In [ ]:
X_missing_train = X_train[['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI']]
X_missing_test = X_test[['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI']]

In [ ]:
knn_imputer = KNNImputer(n_neighbors = 2)
X_filled_train = knn_imputer.fit_transform(X_missing_train)
X_filled_train = pd.DataFrame(X_filled_train,columns = ['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI'])
X_filled_test = knn_imputer.fit_transform(X_missing_test)
X_filled_test = pd.DataFrame(X_filled_test,columns = ['LOTV','NB','DTI','BCSAO','CBCSAO','FTHBI'])

In [ ]:
train_A = X_train.iloc[:,0:13]
train_B = X_train.iloc[:,19:]
X_train = pd.concat([train_A,X_filled_train,train_B], axis=1)
test_A = X_test.iloc[:,0:13]
test_B = X_test.iloc[:,19:]
X_test = pd.concat([test_A,X_filled_test,test_B], axis=1)

In [ ]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
len(np.transpose(X_test))

In [ ]:
df_train = pd.DataFrame(X_train, columns = names_train )
NMONTH_train = pd.DataFrame(Y_train, columns = ['NMONTH'] )
NMONTH_train = NMONTH_train.iloc[:,-1]

In [ ]:
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LinearRegression()
 
acc_score = []
 
for train_index , test_index in kf.split(df_train):
    X_train_train , X_train_test = df_train.iloc[train_index,:],df_train.iloc[test_index,:]
    Y_train , Y_test = NMONTH_train[train_index] , NMONTH_train[test_index]
     
    model.fit(X_train_train,Y_train)
    pred_values = model.predict(X_train_test)
     
    acc = accuracy_score(pred_values , Y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

In [ ]:
Y_train.shape

Transformation

In [ ]:
df_train["NMONTH"]=NMONTH_train.NMONTH
N=df_train.shape[0]
perm=np.random.permutation(range(N))
df_train=df_train.loc[perm]
Ntrain=int(.8*N)
Ntest=N-Ntrain
data_train=df_train.iloc[0:Ntrain]
data_test=df_train.iloc[Ntrain:N]
#data_train.astype('object').dtypes
#data_test.astype('object').dtypes

In [ ]:
formula="NMONTH~0+MRP+CHANNEL+SELLER+ORATE+CRATE+OUPB+OLTERM+ODATE+FPD+LA+TMTLM+RMTM+MDATE+LOTV+NB+DTI+BCSAO+CBCSAO+FTHBI+LP+PTYPE+NUNITS+OCCSTAT+STATE+MSA+ZIP+CLDS+RMI"
Ytrain,Xtrain=ps.dmatrices(formula,data_train)
Ytest,Xtest=ps.dmatrices(formula,data_test)
Ntest=Ytest.shape[0]
reg= LinearRegression().fit(Xtrain,Ytrain)
Ytest_pred=reg.predict(Xtest)
Ytest_pred = np.rint(Ytest_pred)
e=pd.Series((Ytest-Ytest_pred).reshape((Ntest,)))
MAD=e.apply(abs).mean()
print(MAD)
print(Xtrain)

In [ ]:
reg.coef = reg.coef_.reshape(-1)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: x,reg.coef),names_train), reverse=True))

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(Ytest, Ytest_pred)
print('r2 score for perfect model is', r2)

In [ ]:
from sklearn.metrics import accuracy_score
print('The Accuracy of prediction for testing set is:', accuracy_score(Ytest, Ytest_pred))